### Práctica de Formatos de datos: CSV y JSON

#### Ejercicio 1[5 puntos]
Considerar el archivo __Contaminación.csv__. El archivo contiene la información recogida por las estaciones de control de calidad del aire durante el año 2019.

Una descripción detallada de la estructura de los datos se puede encontrar en el siguiente enlace: https://datos.madrid.es/FWProjects/egob/Catalogo/MedioAmbiente/Aire/Ficheros/Interprete_ficheros_%20calidad_%20del_%20aire_global.pdf

La información contenida por columnas es la siguiente:
* __PROVINCIA__: Código numérico que representa la provincia. 
* __MUNICIPIO__: Código numérico que representa el municipio. 
* __ESTACIÓN__: Código numérico de hasta dos dígitos que representa la estación. Consultar ANEXO I de la documentación
* __MAGNITUD__: Código numérico de hasta dos dígitos que representa la magnitud.Consultar ANEXO II de la documentación
* __PUNTO MUESTREO__: Código numérico formado por 3 subcódigos. El primer código representa la estación, el segundo código representa la magnitud, y el tercer código representa la técnica de medida.
* __MES__: Mes de la toma de datos. Valor entre 1 y 12.
* __D01-D31__: Medidas tomadas desde el día D01 hasta el día D31 del mes correspondiente.
* __V01-V31__: Validez de las medidas tomadas desde el día D01 hasta el día D31 del mes correspondiente. Puede valer V(Válido), N(No válido), no se conoce(vacío) o bien otra cadena(se considera incorrecto).


Se pide crear un programa que realice las siguientes operaciones:
* Leer los datos del archivo __Contaminación.csv__
* Transformar los datos leidos de manera que aquellos valores que no sean válidos ("N") su valor sea sustituido por 0.
* Obtener y  mostrar por pantalla para cada tipo de contaminante (columna Magnitud) y para cada estación (columna ESTACIÓN), la suma total de la contaminación que se ha producido durante el año 2019, es decir:

|Estación|Contaminante1| Contaminante2| Contaminante3| ....Contaminanten|
|----|-------------|-------------|-------------|-------------|
|Estación1|Subtotal11 |--- |--- |Subtotal1n|  
|Estación2|--- |--- |--- |--- | 
|   ...|--- |--- |--- |--- |     
|Estaciónn|Subtotal1n |--- |--- |Subtotalnn| 
     
* Generar un documento __Salida.csv__ donde cada línea del documento contiene la siguiente información:
    Estación1, Contaminante1,Subtotal1, Contaminante2,Subtotal2,......................,Contaminanten,Subtotaln
    
    Estaciónm, Contaminante1,Subtotal1, Contaminante2,Subtotal2,......................,Contaminanten,Subtotaln
    
Tanto en lo que mostréis por pantalla como lo que escribáis en el archivo usad los nombres de los contaminantes y de las estaciones que aparecen en los anexos de la documentación indicada más arriba.
    

In [1]:
import csv
import re

diccionarioContaminantes = {1:"Dióxido de Azufre",6:"Monóxido de Carbono",
                 7:"Monóxido de Nitrógeno",8:"Dióxido de Nitrógeno",
                 9:"Partículas <2.5 µm",10:"Partículas < 10µm",12:"Óxidos de Nitrógeno",
                 14:"Ozono",20:"Tolueno",30:"Benceno",35:"Etilbenceno",37:"Metaxileno",
                 38:"Paraxileno",39:"Ortoxileno",42:"Hidrocarburos totales(hexano)",43:"Metano",
                44: "Hidrocarburos no metánicos (hexano)"}

diccionarioEstaciones = {1:"Pº.Recoletos",2:"Glta. de Carlos V", 35:"Pza. del Carmen", 4:"Pza. de España", 39:"Barrio del Pilar",
                    6:"Pza. Dr. Marañón", 7:"Pza. M. de Salamanca", 8:"Escuelas Aguirre", 9:"Pza. Luca de Tena", 38:"Cuatro Caminos",
                     11:"Av. Ramón y Cajal", 12:"Pza. Manuel Becerra", 40:"Vallecas",14:"Pza. Fdez. Ladreda",15:"Pza. Castilla",
                     16:"Arturo Soria",17:"Villaverde Alto",18:"C/ Farolillo",19:"Huerta Castañeda",36:"Moratalaz",21:"Pza. Cristo Rey",
                     22:"Pº. Pontones",23:"Final C/ Alcalá",24:"Casa de Campo",25:"Santa Eugenia",26:"Urb. Embajada (Barajas)",
                     27:"Barajas",47:"Méndez Álvaro",48:"Pº Castellana",49:"Retiro",50:"Pza. Castilla",54:"Ensanche Vallecas",
                     55:"Urb. Embajada (Barajas)",56:"Plaza Elíptica",57:"Sanchinarro",58:"El Pardo",59:"Parque Juan Carlos I",
                     60:"Tres Olivos"}

csvarchivo = open('Contaminación.csv',encoding="utf8",errors='ignore')
entrada = csv.reader(csvarchivo, delimiter=";")
indicesValorar = []
indicesDatos = []


def escribirConsola(tabla):
    cabecera = "Estacion "
    for valor in diccionarioContaminantes.values():
        cabecera += valor + " "
    print(cabecera)
    for fila in tablaFinal:
        estacion = True
        datosEstacion = []
        for dato in fila:
            if estacion:
                datosEstacion.append(diccionarioEstaciones[int(dato)])
                estacion = False
            else:
                datosEstacion.append(dato)
        print(datosEstacion)


        
def escribirCSV(tabla):
    archivoSalida=open("Salida.csv","w")
    salidaEscritor=csv.writer(archivoSalida)
    for fila in tabla:
        listaSalida = []
        contadorFila = 1
        listaSalida.append(diccionarioEstaciones[int(fila[0])])
        for contaminante in diccionarioContaminantes.values():
            listaSalida.append(contaminante)
            listaSalida.append(fila[contadorFila])
            contadorFila+=1
        salidaEscritor.writerow(listaSalida)
    archivoSalida.close()


def parseaNoValidos(indicesValorar, fila):
    for indice in indicesValorar:
        if fila[indice]=="N":
                fila[indice-1] = 0
    return fila

def tratarDatos(i):
    
    fila = []
    contadores=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    indicesContaminantes=[1,6,7,8,9,10,12,14,20,30,35,37,38,39,42,43,44]
    estacion = i[0]
    
    #Para cada dato que tiene la fila se modifica el contador correspondiente
    for j in i[1]:
        contadores[indicesContaminantes.index(int(j[0]))]=j[1]
    
    
    #Se guarda la informacion de toda la linea en la variable fila
    fila.append(estacion)
    #Se recorren todos los contadores para introducir tambien aquellos que no hay muestra
    for z in contadores:
        fila.append(z)
    return fila

def acumulaDatos(contadorActual, indicesDatos, fila):
    for indice in indicesDatos:
        contadorActual += float(fila[indice])
    return contadorActual


contador1 = 0
verifica = 'V'
patron = re.compile(verifica)

magnitudes=[]
estaciones=[]

estacionActual = None
magnitudActual = None
contadorActual = 0
contadoresEstacionActual = []
tablaDatos = []
cogidos = False
for i in entrada:
    if cogidos == False:
        for j in i:
            if(patron.match(j)):
                indicesValorar.append(contador1)
                indicesDatos.append(contador1-1)
            contador1+=1
        cogidos=True
    else:
        i = parseaNoValidos(indicesValorar, i)
        if estacionActual == None:
            estacionActual = i[2]
            magnitudActual = i[3]
            contadorActual = acumulaDatos(contadorActual, indicesDatos, i)
            
        elif estacionActual != i[2]:
            contadoresEstacionActual.append([magnitudActual, contadorActual])
            tablaDatos.append([estacionActual, contadoresEstacionActual])
            estacionActual = i[2]
            magnitudActual = i[3]
            contadorActual = 0
            contadorActual = acumulaDatos(contadorActual, indicesDatos, i)
            contadoresEstacionActual = []
        else:
            if magnitudActual == i[3]:
                contadorActual = acumulaDatos(contadorActual, indicesDatos, i)

            else:
                contadoresEstacionActual.append([magnitudActual, contadorActual])
                magnitudActual = i[3]
                contadorActual = 0
                contadorActual = acumulaDatos(contadorActual, indicesDatos, i)

tablaDatos.append([estacionActual, contadoresEstacionActual])
tablaFinal = []

for i in tablaDatos:
    tablaFinal.append(tratarDatos(i))

    
escribirConsola(tablaFinal)
escribirCSV(tablaFinal)

NameError: name 'tratarDatos' is not defined

#### Ejercicio 2[5 puntos]
Considerar los archivos  __Estaciones.csv__ y __Museos.json__ que incluye información sobre las estaciones de control de la calidad del aire y sobre los museos de Madrid respectivamente.

El objetivo de este ejercicio es utilizar la información obtenida en el ejercicio 1 de forma que para cada uno de los museos que aparece en el archivo dado realice las siguientes operaciones:
* Obtener las 3 estaciones más cercanas a cada museo.
* Obtener y mostrar por pantalla para cada museo la siguiente información donde ValorX es el subtotal del nivel de contaminación más alto de todos los contaminantes medidos en la estaciónX, y TipoContaminanteX es el tipo del contaminante cuyo valor es mostrado en la celda anterior.

|Museo|Estación1|TipoContaminante1| Estación2|TipoContaminante2| Estación3|TipoContaminante3|
|----|-------------|-------------|-------------|----|----|----|
|Museo1|Valor1 |Tipo1 |Valor2|Tipo2|Valor3|  Tipo3 |
|Museo2|--- |--- |--- |--- | --- | --- | 
|   ...|--- |--- |--- |--- | --- | --- |     
|Museon|Valor1 |Tipo1 |Valor2|Tipo2|Valor3|  Tipo3 |

* Generar un documento __Salida.json__ con la siguiente estructura:

Tanto en lo que mostréis por pantalla como lo que escribáis en el archivo usad los nombres de los contaminantes y de las estaciones que aparecen en los anexos de la documentación indicada más arriba.

Para leer los archivos usa un código como el siguiente:

In [ ]:
import json
leer = json.loads(open('Museos.json',encoding="utf8").read())
print(leer)

En los documentos __Estación.csv__ y __Museos.json__ aparece la información de geolocalización en forma de longitud y latitud tanto de las estaciones como de los museos. En este sentido, para calcular la distancia entre dos puntos dadas sus coordenadas se utilizará la siguiente función en Python:

In [3]:
import json
import csv
import re


def convierteADecimal(longitud):
    
    grados = float(re.search("[º°]$", longitud))
    minutos = float(re.search('^[º°].*"$', longitud))
    segundos = float(re.search('^\'.*"$', longitud))
    print(longitud)
    datoActual = ""
    positivo = True
    for caracter in longitud:
        if caracter == "°" or caracter == "º":
            datoActual = datoActual.strip()
            grados = float(datoActual)
            datoActual = ""
        elif caracter == "'":
            print(datoActual)
            datoActual = datoActual.strip()
            minutos = float(datoActual)
            datoActual = ""
        elif caracter == '"':
            datoActual = datoActual.strip()
            segundos = float(datoActual)
            datoActual = ""
        elif caracter == "O" or caracter == "S":
            positivo = False
        elif caracter == "E" or caracter == "N":
            positivo = True
        elif caracter != " ":
            datoActual += caracter
    resultado = (grados+(minutos/60)+(segundos/3600))
    if not positivo:
        resultado = -resultado
    return resultado


csvarchivo = open('Estaciones.csv')
entrada = csv.reader(csvarchivo, delimiter=";")


leer = json.loads(open('Museos.json',encoding="utf8").read())

estaciones = []
estacionActual = []
primeraLinea = True



for museo in leer["@graph"]:
    print(museo["title"])
    print(museo["location"]["latitude"])
    print(museo["location"]["longitude"])


Casa Museo Lope de Vega
40.414358466555235
-3.6974741545860015
Casita - Museo del Ratón Pérez
40.41704834412296
-3.7055288844865233
Castillo de la Alameda
40.45835460563491
-3.5932903820278788
Casón del Buen Retiro
40.415263749471535
-3.689232811717636
Centro Social y Cultural La Casa Encendida
40.40595710517701
-3.699779793558763
Ermita de San Antonio de la Florida (Museo)
40.425256682855846
-3.725725718176111
Espacio Fundación Telefónica
40.42041458612889
-3.7016034186955076
Fundación AMYC (Arte Moderno y Contemporáneo). Casa - Museo Fuente del Rey
40.45546180290319
-3.7705836264842576
Monasterio de las Descalzas Reales
40.418233359889484
-3.706213134639163
Museo ABC
40.42741121610518
-3.709539260414957
Museo Africano Mundo Negro
40.449464654033605
-3.65149172790308
Museo Arqueológico Nacional
40.42327763947072
-3.688418705931963
Museo Casa de la Moneda
40.42259797372905
-3.6691502061733674
Museo Cerralbo
40.42376732078657
-3.7146170388153497
Museo EMT
40.480173612860305
-3.683159545

KeyError: 'location'

#### Normas de entrega

* Fecha tope de entrega: 03/10/2019
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre FormatosI_GrupoX donde X será el número de grupo correspondiente.